[View in Colaboratory](https://colab.research.google.com/github/NaNkotsukan/deepLearningWithChainer/blob/master/AutoEncoder.ipynb)

## 環境設定

In [0]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80 chainer matplotlib
!nvidia-smi

# AutoEncoder を使った異常検知
AutoEncoderとは自己符号化器のことである．
入力と出力を同じにして中間層を絞ることで特徴量を抽出することが出来る．

In [0]:
from chainer import Chain, serializers, computational_graph
from chainer import Variable as V
import chainer.functions as F
import chainer.links as L
import chainer.computational_graph as C
from chainer.optimizers import Adam, SGD
import chainer

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image, display

import os
import numpy as np
import cupy as cp

%matplotlib inline

### データセットの読み込み

In [0]:
train, test = chainer.datasets.get_mnist(ndim=3)
train_x = train._datasets[0]
train_t = train._datasets[1]
test_x = test._datasets[0]
test_t = test._datasets[1]

print(train_x.shape)
print(train_t.shape)
print(test_x.shape)
print(test_t.shape)

In [0]:
def show_img(img, title=None):
    fig = plt.figure(figsize=(15, 20))
    for i in range(len(img)):
        if img[i].ndim==3: x = img[i][0]
        else: x = img[i]
        a = fig.add_subplot(1, len(img), i+1)
        a.imshow(x)
        plt.xlim((0, 28))
        plt.ylim((28, 0))
        plt.tick_params(labelbottom=False, labelleft=False)
        if title != None:
            plt.title(title[i], fontsize=15)
        plt.grid()
    plt.show()

## モデルを定義する

In [0]:
class AutoEncoder(Chain):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        with self.init_scope():
            self.l0 = L.Linear(784, 500)
            self.l1 = L.Linear(500, 300)
            self.l2 = L.Linear(300, 200)
            self.l3 = L.Linear(200, 100)
            self.l4 = L.Linear(100, 50)
            self.l5 = L.Linear(50, 100)
            self.l6 = L.Linear(100, 200)
            self.l7 = L.Linear(200, 300)
            self.l8 = L.Linear(300, 500)
            self.l9 = L.Linear(500, 784)


    def __call__(self, x):
        h = F.relu(self.l0(x))
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = F.relu(self.l3(h))
        h = F.relu(self.l4(h))
        h = F.relu(self.l5(h))
        h = F.relu(self.l6(h))
        h = F.relu(self.l7(h))
        h = F.relu(self.l8(h))
        h = self.l9(h)
        h = F.reshape(h, (-1, 1, 28, 28))
        return h

## trainingクラスの定義

In [0]:
class Training:
    def __init__(self, model):
        self.model = model
        self.optimizer = Adam()
        self.optimizer.setup(self.model)
        self.model.to_gpu()

    def test(self, z):
        y = self.model(z).data.get()
        show_img(y)

    def batch(self, x):
        y = self.model(x)
        loss = F.mean_squared_error(x, y)
        
        self.model.cleargrads()
        loss.backward()
        self.optimizer.update()
        
        return loss.data

    def training(self, epoch, batchsize):
        for i in range(epoch):
              index = np.random.permutation(len(train_x))
              for j in range(0, len(train_x), batchsize):
                x = cp.asarray(train_x[index[j:j+batchsize]]).astype(np.float32).reshape(batchsize, 1, 28, 28)
                loss = self.batch(x)
                if j%10000==0:
                    print(f"epoch:{i:3d} batch:{j:6d} loss:{float(loss):0.4f}")
              x = cp.asarray(test_x[:10])
              self.test(x)
              



In [0]:
model = AutoEncoder()
hoge = Training(model)
hoge.training(epoch=10, batchsize=50)

## 異常の有る画像を生成する関数の定義

In [0]:
def noise0(x):
    y = np.array(x)
    a = np.random.uniform(0., 1., test.shape)<0.01
    y[a] = 1-y[a]
    return y

def noise1(x):
    y = np.array(x)
    y[:,:,:,15:17] = 1
    return y

In [0]:
original = test_x[:10]
test = noise0(original)
# test = noise1(test)

result = model(cp.asarray(test)).data.get()
for i in range(10):
    show_img([original[i], test[i], result[i], np.absolute(test[i]-result[i])], ["original", "x", "y", "diff"])
